<div>
     <div>
        <img src="./report/isel_logo.png" width="400" height="400" align="left">
    </div>
    <div>
        <h2>Área Departamental de Engenharia de Eletrónica e Telecomunicações e de Computadores</h2>
        <p>Trabalho prático 2</p>
        <p>Autor:	44598	André L. A. Q. de Oliveira</p>
        <p>Unidade Curricular Compressão de Sinais Multimédia</p>
        <p>Professor: André Lourenço</p>
        <p>9 – Maio – 2021</p>
    </div>
</div>

### <a id="index"></a>

# Index
- [Codificação de Huffman](#codificacao_huffman)
- [I/O Utilities](#io_utilities)
    - [pad_bits](#pad_bits)
    - [to_binary_list](#to_binary_list)
    - [InputBitReader](#input_bit_reader)
- [Tabela de Huffman](#tabela_huffman)
    - [bytes_frequency](#bytes_frequency)
    - [huff_node](#huff_node)
    - [create_huff_tree](#create_huff_tree)
    - [huff_tree_encode](#huff_tree_encode)
    - [huff_tree_decode](#huff_tree_decode)
    - [huff_tree2huff_dictionary](#huff_tree2huff_dictionary)
    - [gen_huff_table](#gen_huff_table)
- [Codificador de Huffman](#codificador_huffman)
    - [encode_huff](#encode_huff)
- [Descodificador de Huffman](#descodificador_huffman)
    - [decode_huff](#decode_huff)
- [Escreve para ficheiro](#escrever_ficheiro)
    - [write2file](#write2file)
- [Ler ficheiro](#ler_ficheiro)
    - [read2array](#read2array)
- [Testes](#testes)

<a id="codificacao_huffman"></a>

# Codificação de Huffman

A codificação de Huffman é um método de compressão, desenvolvido em 1952 por  David A. Huffman, que usa as probabilidades de ocorrência dos símbolos nde um conjunto de dados a ser comprimido para determinar códigos binários de tamanho variável para cada símbolo.

Uma árvore binária completa, chamada de árvore de Huffman é construída recursivamente a partir da junção dos dois símbolos de menor probabilidade, que são então somados em símbolos auxiliares que são depois recolocados no conjunto de símbolos. O processo termina quando todos os símbolos forem unidos em símbolos auxiliares, formando uma árvore binária. A árvore é então percorrida, atribuindo-se valores binários de 1 ou 0 para cada aresta, e os códigos são gerados a partir desse percurso.

O resultado do algoritmo de Huffman pode ser visto como uma tabela de códigos de tamanho variável para codificar um símbolo. Os símbolos mais comuns são geralmente representados usando-se menos dígitos que os símbolos que aparecem com menos frequência.


Para a string **"go go gophers"**, seria gerada a seguinte árvore de Huffman e respetiva tabela:

![huff-table-example](./report/huff-table-example.PNG)

A string seria codificada como: 000 001 111 000 001 111 000 001 010 011 100 101 110. Neste caso seriam utilizados três bits por caractere (em vez de oito bits por caractere como acontece no ASCII), a string **"go go gophers"** após codificação usaria um total de 39 bits em vez de 104 bits.



# Importar bibliotecas

In [3]:
import os
from time import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json

cwd = os.getcwd() # current work diretory

# Importar dados para teste

In [37]:
test1 = np.frombuffer('otorrinolaringologista'.encode('utf-8'), dtype='uint8')
test2 = np.frombuffer('go go gophers'.encode('utf-8'), dtype='uint8')

dec_universal_IDH_txt = np.fromfile(f"{cwd}/data/DecUniversalDH.txt", dtype='uint8')
dec_universal_IDH_pdf = np.fromfile(f"{cwd}/data/DecUniversalDH.pdf", dtype='uint8')
henry_mancini_mp3 = np.fromfile(f"{cwd}/data/HenryMancini-PinkPanther30s.mp3", dtype='uint8')
henry_mancini_mid = np.fromfile(f"{cwd}/data/HenryMancini-PinkPantherC.mid", dtype='uint8')
lena_color = np.fromfile(f"{cwd}/data/LenaColor.tif", dtype='uint8')
lena_gray = np.fromfile(f"{cwd}/data/LenaGray.tif", dtype='uint8');

<a id="io_utilities"></a>

# I/O Utilities

<a id="pad_bits"></a>

## pad_bits

Extende o número de zeros a uma sequência de bits, para permitir codificação de tamanho fixo. Os zeros são adicionados nas posições de bit mais significantes

In [5]:
def pad_bits(bits, n):
    # prefix string of bits with enough zeros to reach n digits
    if isinstance(bits, np.ndarray):
        if(n - len(bits) > 0):
            return np.pad(bits, (n - len(bits), 0))
        else:
            return bits
    else:
        return ([0] * (n - len(bits)) + bits)

<a id="to_binary_list"></a>

## to_binary_list
Converte um número inteiro na menor sequência de bits que o representa.

In [6]:
def to_binary_list(n):
    # Convert integer into a list of bits
    return [n] if (n <= 1) else to_binary_list(n >> 1) + [n & 1]

    # return [int(i) for i in list('{0:0b}'.format(n))]

<a id="input_bit_reader"></a>

## InputBitReader

Para realizar compressão e descompressão com eficácia, é necessesário manipular os fluxos de dados como um fluxo de bits individuais. 

In [7]:
class InputBitReader(object): 
    def __init__(self, bit_seq): 
        self.bit_seq = bit_seq
        self.size = len(bit_seq)
        self.bits_read = 0
        self.buffer = []

    def read_bit(self):
        if self.bits_read < self.size:
            return self.read_bits(1)[0]
        else:
            return None

    def read_bits(self, n):
        self.__flush()
        if self.bits_read < self.size:
            self.buffer = pad_bits(self.bit_seq[self.bits_read:(self.bits_read + n)], n)
            self.bits_read += n
        return self.buffer
    
    def read_byte(self):
        if self.bits_read < self.size:
            byte = ''.join(list(map(str, self.read_bits(8))))
            return int(byte, 2)
        else:
            return None

    def __flush(self):
        self.buffer = []

 <a id="tabela_huffman"></a>

# Tabela de Huffman

A codificação de Huffman é um método de compressão que usa as probabilidades de ocorrência dos símbolos no conjunto de dados a ser comprimido para determinar códigos de tamanho variável para cada símbolo.



 <a id="bytes_frequency"></a>

## bytes_frequency


Lê um ficheiro, byte a byte, e retorna um dicionário com par chave-valor : símbolo-frequência, onde cada símbolo terá como respondência a sua frequência no ficheiro. 

In [9]:
# return dicionary {byte : frequency}
def bytes_frequency(file):
    d = dict()
    for byte in file:
        d[byte] = d.setdefault(byte, 0) + 1
    return d

 <a id="huff_nome"></a>

## huff_node

Classe que representa um nó de Huffman. Cada nó contêm a seguinte informação:
* o símbolo
* a frequência do símbolo
* uma ligação para a esquerda e para a direita para os seus nós filhos
* o valor de huffman atribuído quando o nó toma uma direção

In [55]:
# Huffman Node
class huff_node:
    def __init__(self, symbol, freq, left = None, right = None):
        # symbol
        self.symbol = symbol
        # frequency of symbol
        self.freq = freq
        # node left of current node
        self.left = left
        # node right of current node
        self.right = right

 <a id="create_huff_tree"></a>

## create_huff_tree

A árvore de Huffman é construída recursivamente a partir da junção dos dois símbolos de menor probabilidade, que são então somados em símbolos auxiliares e estes símbolos auxiliares recolocados no conjunto de símbolos. O processo termina quando todos os símbolos forem unidos em símbolos auxiliares, formando uma árvore binária.

1. Com o valor de cada chave única presente no dicionário, são criados nós e colocados numa lista;
2. São retirados os dois símbolos com menor frequência da lista, atribuindo-lhes o valor de 0 ou 1, e mergem-se esses dois símbolos num só, somando as suas freqûencias; 
3. O novo nó é adicionado a lista;
4. O prodecimento repete-se até enquanto o número de nós for superior a 1;
5. A função retorna o nó raiz.

In [11]:
# Create HuffmanTree from dictionary of {symbol : frequency}
def create_huff_tree(dictionary):
    tree = []
    for symb, freq in dictionary.items():
        tree.append(huff_node(symb, freq))
    
    while len(tree) > 1:
        tree = sorted(tree, key=lambda n: n.freq)
        # pop the 2 smallest nodes
        left = tree.pop(0)
        right = tree.pop(0)
        # combine the 2 smallest nodes to create new node as their parent
        new_node = huff_node(str(left.symbol) + str(right.symbol), left.freq + right.freq, left, right)
        tree.append(new_node)
        
    return tree[0]

<a id="huff_tree2huff_dictionary"></a>

## huff_tree2huff_dictionary

A partiz de uma árvore de Huffman gera uma tabela de Huffman. A árvore é percorrida, atribuindo-se valores binários de 1 ou 0 para cada aresta, e os códigos são gerados a partir desse percurso.

In [12]:
def huff_tree2huff_dictionary(node):
    d = dict()
    huff_tree2huff_dictionary_aux(node, d)
    return d

In [13]:
def huff_tree2huff_dictionary_aux(node, dictionary, value = ""):
    if(node.left):
        huff_tree2huff_dictionary_aux(node.left, dictionary, value + "0")
    if(node.right):
        huff_tree2huff_dictionary_aux(node.right, dictionary, value + "1")
    # if node is leaf
    if(not node.left and not node.right):
        dictionary[node.symbol] = value

In [45]:
d = bytes_frequency('go go gophers')
huffman_tree = create_huff_tree(d)
d = huff_tree2huff_dictionary(root1)
print(d)

{'g': '00', 'o': '01', 's': '100', ' ': '101', 'p': '1100', 'h': '1101', 'e': '1110', 'r': '1111'}


<a id="huff_tree_encode"></a>

## huff_tree_encode

Além de compactar um ficheiro, é necessário também armazenar um cabeçalho no arquivo compactado que será utilizado pelo programa de descompactação. Em suma, é necessário de alguam forma, armazenar a árvore utilizada para compactar o ficheiro original. Esta necessidade deve-se ao facto de o programa de descompressão precisa também dessa mesma árvore para decodificar os dados.

Para armazenar a árvore de huffman no cabeçalho do ficheiro, recore-se a estratégia de pesquisa em árvore "post-order transversel", para assinalar cada nó visitado. Ao encontrar um nó folha, é escrito o valor 1, seguido pelo símbolo do nó folha. Ao encontrar um nó wue não seja folha, é escrito o valor um 0.

Considerando a mesma string utilizanda anteriormente como exemplo, **"go go gophers"**, a informação do cabeçalho ficaria expressa sepla seguinte codificação: **"1g1o01s1 01e1h01p1r0000"**.

In [14]:
def huff_tree_encode(node):
    bits = []
    huff_tree_encode_postorder(node, bits)
    
    huff_table = np.array([], dtype='uint8')
    for bit in bits:
        huff_table = np.append(huff_table, bit)
    huff_table = np.packbits(huff_table)
    
    return huff_table

In [15]:
def huff_tree_encode_postorder(node, bits):
    if (node.left):
        huff_tree_encode_postorder(node.left, bits)
    if (node.right):
        huff_tree_encode_postorder(node.right, bits)
    # if node is leaf
    if (not node.left and not node.right):
        bits.append(1)
        bits.append(np.unpackbits(node.symbol))
    else:
        bits.append(0)

In [59]:
def huff_tree_encode_postorder_ascii_example(node, symbols):
    if (node.left):
        huff_tree_encode_postorder_ascii_example(node.left, symbols)
    if (node.right):
        huff_tree_encode_postorder_ascii_example(node.right, symbols)
    # if node is leaf
    if (not node.left and not node.right):
        symbols.append(1)
        symbols.append(node.symbol)
    else:
        symbols.append(0)

d = bytes_frequency('go go gophers')
huffman_tree = create_huff_tree(d)
huff_table = []
huff_tree_encode_postorder_ascii_example(huffman_tree, huff_table)
print(''.join(list(map(str,huff_table))))

1g1o01s1 01p1h01e1r0000


<a id="huff_tree_decode"></a>

## huff_tree_decode

A construção da árvore de Huffman a partir do cabeçalho, é realizada com recurso a um stack. A informação do cabeçalho deve ser lida bit a bit. Quando se lê um bit com o valor 1, significa que se está perante um nó do tipo folha, então é lido o próximo byte e coloca-se o símbolo no stack. Quando um bit com o valor 0 é lido, se a pilha contém apenas um elemento, então toda a árvore de Huffman está construída. Caso contrário, deve haver mais de um elemento na pilha, então são retirados os dois primeiros elementos da pilha. O primeiro elemento do stack é um novo nó direito, e o segundo elemento do stack é um novo nó esquerdo. Um o nó pai é criado com os filhos nó esquerdo e direito recém-criados, e é colocado depois no stack.

In [31]:
def huff_tree_decode(huff_table):
    ibr = InputBitReader(np.unpackbits(huff_table))
    tree = []
    bits_read = 0
    
    while True:
        if (ibr.read_bit() == 1):
            byte = ibr.read_byte()
            tree.append(huff_node(byte, 0, None, None))
        else:
            # if tree contains only 1 element, then its complete
            if len(tree) == 1:
                break
            right = tree.pop()
            left = tree.pop()
            tree.append(huff_node(0, 0, left, right))
            
    return tree[0]

<a id="gen_huff_table"></a>

## gen_huff_table

Função que agrega todas as chamadas para gerar a tabela de Huffman.

In [18]:
def gen_huff_table(file):
    d = bytes_frequency(file)
    huff_tree = create_huff_tree(d)
    huff_dictionary = huff_tree2huff_dictionary(huff_tree)
    huff_table = huff_tree_encode(huff_tree)
    return huff_table, huff_dictionary

In [60]:
# generate huff table
encoded_table, huff_dictionary = gen_huff_table(test1)
print(huff_dictionary, '\n')

# re-construct huffman tree from huff table
decoded_huffman_tree = huff_tree_decode(encoded_table)
# get huffman dictionary from huff table
new_huff_dictionary = huff_tree2huff_dictionary(decoded_huffman_tree)
print(new_huff_dictionary, '\n')

{97: '000', 103: '001', 111: '01', 114: '100', 105: '101', 115: '1100', 116: '1101', 110: '1110', 108: '1111'} 

{97: '000', 103: '001', 111: '01', 114: '100', 105: '101', 115: '1100', 116: '1101', 110: '1110', 108: '1111'} 



 <a id="codificador_huffman"></a>

# Codificador de Huffman

<a id="encode_huff"></a>

## encode_huff

Função que dada uma mensagem e uma tabela de Huffman, codifica uma mensagem.

In [20]:
def encode_huff(file, huff_dictionary):
    biq_seq = ""
    for byte in file:
        biq_seq += huff_dictionary[byte]
    return list(map(int, biq_seq))

In [21]:
biq_seq = encode_huff(test1, huff_dictionary)
print("biq_seq \n", biq_seq)

biq_seq 
 [0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0]


 <a id="descodificador_huffman"></a>

# Descodificador de Huffman

<a id="decode_huff"></a>

## decode_huff

Função que a partir de uma sequência de bits (mensagem codificada) e uma tabela de huffman, retorne uma mensagem descodificada.

In [29]:
def decode_huff (bit_seq, huff_table):
    huff_tree = huff_tree_decode(huff_table)
    huff_dictionary = huff_tree2huff_dictionary(huff_tree)
    inverted_huff_dictionary = dict(map(reversed, huff_dictionary.items()))
    buffer = ""
    output = []
    for bit in bit_seq:
        buffer += str(bit)
        if buffer in inverted_huff_dictionary:
            output.append(inverted_huff_dictionary[buffer])
            buffer = ""
    return bytearray(output)

In [23]:
decoded_msg = decode_huff(biq_seq, huff_table)
print(np.unpackbits(decoded_msg))
print((str(decoded_msg, 'utf-8')))

[0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 0 0 1 1 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 1 1 0
 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 0 1
 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 0 1 1
 1 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 1 1 0 0 1 1 1 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1]
otorrinolaringologista


 <a id="escrever_ficheiro"></a>

# Escrever para ficheiro

A escrita para um ficheiro, é conseguida de maneira trivial

<a id="write2file"></a>

## write2file

In [39]:
def write2file(huff_table, bit_seq, filename): 
    
    # calculate size of huffman table
    t_size = pad_bits(to_binary_list(len(huff_table)), 8)
    table_size = np.packbits(t_size)
    
    # calculate size of data
    d_size = pad_bits(to_binary_list(len(bit_seq)), 8)
    data_size = np.packbits(d_size)
    
    output = np.array([], dtype='uint8')
    ### header info
    # byte[0] : 
    #  size of huffman table
    output = np.append(output, table_size)
    # byte[1] - byte[table_size] : 
    #  huffman table
    output = np.append(output, huff_table)
    
    ### data info
    # byte[table_size + 1] : 
    #  size of data
    output = np.append(output, data_size)
    # byte[table_size + 1] - eof : 
    #   data
    byte_seq = np.packbits(bit_seq)
    output = np.append(output, byte_seq)
    
    np.save(f"{cwd}/data/{filename}", output)

In [25]:
huff_table, huff_dictionary = gen_huff_table(test2)
bit_seq = encode_huff(test2, huff_dictionary)

write2file(huff_table, bit_seq, "test")

w-huff_table [179 219 215  57   2 225 104  89 110  64]
w-data_size [37]
w-bit_seq [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0]
w-byte_seq [ 26  52 115 123 224]


 <a id="ler_ficheiro"></a>

# Ler ficheiro

In [26]:
def read2array(filename):
    # size of huffman table
    table_size = filename[0]
    # huffman table
    huff_table = filename[1:(table_size + 1)]
    # size of data
    data_size = filename[(table_size + 1)]
    # data
    byte_seq = filename[(table_size + 1 + 1):]
    # get bit array sequence from byte array
    bit_seq = np.unpackbits(byte_seq)
    # from the bit sequence we are only interested on the first data_size bits
    # so we ignore the rest
    bit_seq = bit_seq[:data_size]
    # decode bit sequence using the huff table
    return decode_huff(bit_seq, huff_table)

In [27]:
encoded_file = np.load(f"{cwd}/data/test.npy")
byte_seq = read2array(encoded_file)
print("unpack", np.unpackbits(byte_seq))
print((str(byte_seq, 'utf-8')))

table size 10
huff table [179 219 215  57   2 225 104  89 110  64]
data size 37
byte seq [ 26  52 115 123 224]
bit_seq [0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0
 0 0 0]
bit seq 37 [0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0]
unpack [0 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 0 1
 1 1 1 0 0 1 0 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1
 1 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 1 1]
go go gophers


 <a id="testes"></a>

# Testes

In [43]:
t_start = time()
huff_table, huff_dictionary = gen_huff_table(dec_universal_IDH_txt)
t_end = time()
print("dec_universal_IDH_txt:")
print("file size [bytes]: ", os.stat(f"{cwd}/data/DecUniversalDH.txt").st_size)
print(f"time to create huff table & dictionary [seconds]: {t_end - t_start}")
print()

t_start = time()
huff_table, huff_dictionary = gen_huff_table(dec_universal_IDH_pdf)
t_end = time()
print("dec_universal_IDH_pdf:")
print("file size [bytes]: ", os.stat(f"{cwd}/data/DecUniversalDH.pdf").st_size)
print(f"time to create huff table & dictionary [seconds]: {t_end - t_start}")
print()

t_start = time()
huff_table, huff_dictionary = gen_huff_table(henry_mancini_mp3)
t_end = time()
print("henry_mancini_mp3:")
print("file size [bytes]: ", os.stat(f"{cwd}/data/HenryMancini-PinkPanther30s.mp3").st_size)
print(f"time to create huff table & dictionary [seconds]: {t_end - t_start}")
print()

t_start = time()
huff_table, huff_dictionary = gen_huff_table(henry_mancini_mid)
t_end = time()
print("henry_mancini_mid:")
print("file size [bytes]: ", os.stat(f"{cwd}/data/HenryMancini-PinkPantherC.mid").st_size)
print(f"time to create huff table & dictionary [seconds]: {t_end - t_start}")
print()

t_start = time()
huff_table, huff_dictionary = gen_huff_table(lena_color)
t_end = time()
print("lena_color:")
print("file size [bytes]: ", os.stat(f"{cwd}/data/LenaColor.tif").st_size)
print(f"time to create huff table & dictionary [seconds]: {t_end - t_start}")
print()

t_start = time()
huff_table, huff_dictionary = gen_huff_table(lena_gray)
t_end = time()
print("lena_gray:")
print("file size [bytes]: ", os.stat(f"{cwd}/data/LenaGray.tif").st_size)
print(f"time to create huff table & dictionary [seconds]: {t_end - t_start}")
print()

dec_universal_IDH_txt:
file size [bytes]:  11930
time to create huff table & dictionary: 0.00698089599609375

dec_universal_IDH_pdf:
file size [bytes]:  17524
time to create huff table & dictionary: 0.024449586868286133

henry_mancini_mp3:
file size [bytes]:  236925
time to create huff table & dictionary: 0.06876254081726074

henry_mancini_mid:
file size [bytes]:  48049
time to create huff table & dictionary: 0.015991687774658203

lena_color:
file size [bytes]:  786572
time to create huff table & dictionary: 0.20046234130859375

lena_gray:
file size [bytes]:  210122
time to create huff table & dictionary: 0.052839040756225586

